In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

This notebook is a tutorial jupyter notebook of the package ``structure_factor`` version 2.0.1 designed to study the structure factor of a point process and test its hyperuniformity. 
We use the Ginibre point process as a running example. It is convenient, since its pair correlation function, structure factor, and hyperuniformity class are analytically known.

# 1- Installation

If the package is not already installed/updated on your local machine you can simply use the following first line/second for the installation/update.

In [ ]:
# !pip install structure-factor
# !pip install structure-factor --upgrade

# 2- Creat a PointPattern object 

To create an object of type ``PointPattern`` from a point process, we need a sample of points generated from the point process, the observation window which must be ``BallWindow`` or  ``BoxWindow``, and the intensity of the point process (optional). 
A ``PointPattern`` from the Poisson, Thomas, or Ginibre point process could be created using the module ``point_process`` of this package. 
For more details, we refer to the documentation of this project https://for-a-few-dpps-more.github.io/structure-factor/
In what follows we will be loading a Ginibre PointPattern (instead of creating one) using ``load_data`` from the module ``data``.

In [ ]:
# Load Ginibre PointPattern 
from structure_factor.data import load_data
ginibre_pp_ball = load_data.load_ginibre()
# Plot
ginibre_pp_ball.plot()

# 3- Approximate the structure factor

Actually, the package gathers 3 direct spectral estimators, and 3 isotropic estimators of the structure factor of a ``PointPattern``.

## 3.1- Using a direct spectral estimator

The direct spectral estimators approximate the structure factor of a stationary point process observed in a ``BoxWindow``.
As the ``ginibre_pp_ball`` is a ``PointPattern`` having ``BallWindow`` we will first restrict it to a new ``PointPattern`` with ``BoxWindow``.

In [ ]:
from structure_factor.spatial_windows import BoxWindow

# Restrict ginibre_pp_ball to a BoxWindow
l = 70 # length side of the BoxWindow
box_window = BoxWindow(bounds=[[-l/2, l/2], [-l/2, l/2]]) # BoxWindow
ginibre_pp_box = ginibre_pp_ball.restrict_to_window(box_window) # Restrict to box_window
# Plot
ginibre_pp_box.plot()


As we have the adequate ``PointPattern`` we will initialize StructureFactor and then use the estimators progressively.

In [ ]:
from structure_factor.structure_factor import StructureFactor

# initialize ``StructureFactor`` on ginibre_pp_box
sf_ginibre_box = StructureFactor(ginibre_pp_box) 

### 3.1.1- Scattering intensity

The scattering intensity $\widehat{S}_{\mathrm{SI}}$ is an estimator of the structure factor $S$.
It could be evaluated on arbitrary wavevectors, or allowed wavevectors (reduce bias).
It has also 2 debiased versions directly/undirectly.

In [ ]:
# Scattering intensity on arbitrary wavevectors

# Construct a grid of wavevectors
k_max=8 # Threshold on the maximum wavenumber
x = np.linspace(0, k_max, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel())) # Wavevectors
# Scattering intensity on k 
k, s_si_k = sf_ginibre_box.scattering_intensity(k=k, # Wavevectors
                                                debiased=False # # undebiased estimator
                                               )

# Plot
import matplotlib.pyplot as plt

fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_box.plot_spectral_estimator(k, s_si_k, 
                                       axes=axis, # Plot axis
                                       plot_type="radial", # Plot type (radial, all or imshow)
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=60, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$", # Label
                                       # file_name="ginibre_s_si.pdf" # Save the figure
                                      )

We can see the relevant bias at small wavenumbers. To deal with this bias we can use one of the following three debiased versions.

In [ ]:
# Scattering intensity directly debiased 

k_max=8 # Threshold on the maximum wavenumber
# Scattering inetsnity on allowed_k 
allowed_k, s_si_allowed_k = sf_ginibre_box.scattering_intensity(k_max=k_max)

# Plot
sf_ginibre_box.plot_spectral_estimator(allowed_k, s_si_allowed_k, 
                                       plot_type="all", # Plot type  
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=60, # Regularization of the result
                                       label=r"$\widehat{S}$",
                                      # file_name="ginibre_s_si.pdf" # Save the figure
                                      )

In [ ]:
# Scattering intensity directly debiased on arbitrary wavevectors

# Construct grid of wavevectors
k_max=8 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel())) # Wavevectors
# Scattering intensity on k 
k, s_si_k_dd = sf_ginibre_box.scattering_intensity(k=k, # Wavevectors
                                                   debiased=True, # Debiased estimator
                                                   direct=True # Directly debiased
                                                  )

# Plot
import matplotlib.pyplot as plt

sf_ginibre_box.plot_spectral_estimator(k, s_si_k_dd,  
                                       plot_type="all", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$",
                                      # file_name="ginibre_s_si.pdf" # Save the figure
                                      )

In [ ]:
# Scattering intensity undirectly debiased on arbitrary wavevectors

# Construct grid of wavevectors
k_max=8 # Threshold on the maximum wavenumber
x = np.linspace(0, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel())) # Wavevectors
# Scattering intensity on k 
k, s_si_k_ud = sf_ginibre_box.scattering_intensity(k=k, # Wavevectors
                                                   debiased=True, # Debiased estimator
                                                   direct=False # Undirectly debiased
                                                  )

# Plot
import matplotlib.pyplot as plt

sf_ginibre_box.plot_spectral_estimator(k, s_si_k_ud, 
                                       positive=True, # Plot only positive values
                                       plot_type="radial", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$",
                                      # file_name="ginibre_s_si.pdf" # Save the figure
                                      )

### 3.1.2- Scaled tapered periodogram with sine taper

The scaled tapered periodogram $\widehat{S}_{\mathrm{TP}}$ is an estimator of the structure factor $S$ depending on a taper function.
It has 2 debiased versions $\widehat{S}_{\mathrm{DDTP}}$, and $\widehat{S}_{\mathrm{UDTP}}$ (directly/undirectly).
In what follow we will be using the first taper of the family of sinusoidal tapers available in the module ``tapers``.
To use a new taper see the example in the documentation.

In [ ]:
# Scaled tapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(0, k_max, 250)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# First taper of the family of sinusoidal tapers
p = [1, 1]
taper = SineTaper(p)
# Scaled tapered periodogram
s_tp = sf_ginibre_box.tapered_periodogram(k, # Wavevectors
                                          taper, # Taper
                                          debiased=False # Undebiased estimator
                                         )

# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_tp, 
                                       plot_type="radial", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$")

We can see the relevant bias at small wavenumbers. To deal with this bias we can use one of the following twow debiased versions.

In [ ]:
# Directly debiased scaled tapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# First taper of the family of sinusoidal tapers
p = [1, 1]
taper = SineTaper(p)
# Directly debiased scaled tapered periodogram
s_ddtp = sf_ginibre_box.tapered_periodogram(k, # Wavevector
                                            taper, # Taper
                                            debiased=True, # Debiased estimator
                                            direct=True # Directly debiased
                                           )

# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_ddtp, 
                                       plot_type="all", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$")

In [ ]:
# Undirectly debiased scaled tapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# First taper of the family of sinusoidal tapers
p = [1, 1]
taper = SineTaper(p)
# Undirectly debiased scaled tapered periodogram
s_udtp = sf_ginibre_box.tapered_periodogram(k, # Wavevectors
                                            taper, # Taper
                                            debiased=True, # Debiased estimator
                                            direct=False # Undirectly debiased
                                           )

# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_udtp, 
                                       plot_type="all", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$",
                                      # file_name="ginibre_s_udtp.pdf" # Save the figure
                                      )

### 3.1.3- Scaled multitapered periodogram

The scaled multitapered periodogram $\widehat{S}_{\mathrm{MTP}}$ is an estimator of the structure factor $S$ depending on a family of tapers.
It has 2 debiased versions $\widehat{S}_{\mathrm{DDMTP}}$, and $\widehat{S}_{\mathrm{UDMTP}}$ (directly/undirectly).
In what follow we will be using the first 4 tapers of the family of sinusoidal tapers available in the module ``tapers``.
To use a new family of tapers, follow the example present in the documentation.

In [ ]:
# Scaled multitapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(0, k_max, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# First 4 tapers of the family of sinusoidal tapers
p_list = [[1, 1], [1,2], [2,1], [2,2]] # List of p 
taper_list = [SineTaper(p) for p in p_list] # List of tapers
# Scaled multitapered periodogram
s_tp = sf_ginibre_box.multitapered_periodogram(k, # Wavevectors
                                               tapers=taper_list, # List of tapers
                                               debiased=False # Undebiased estimator
                                              )

# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_tp, 
                                       plot_type="radial", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$")

We can see the relevant bias at small wavenumbers. To deal with this bias we can use one of the following twow debiased versions.

In [ ]:
# Directly debiased scaled multitapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))

# Directly debiased scaled multitapered periodogram
# By default the method ``multitapered_periodogram`` use the first 4 tapers of the family of sinusoidal tapers.
s_ddtp = sf_ginibre_box.multitapered_periodogram(k, # Wavevectors
                                                 debiased=True, # Debiased estimator
                                                 direct=True # Directly debiased
                                                ) 
# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_ddtp, 
                                       plot_type="all", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$")

In [ ]:
# Undirectly debiased scaled multitapered periodogram
from structure_factor.tapers import SineTaper
 
k_max = 8 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))

# Undirectly debiased scaled multitapered periodogram
# By default the method ``multitapered_periodogram`` use the first 4 tapers of the family of sinusoidal tapers.
s_udtp = sf_ginibre_box.multitapered_periodogram(k, # Wavevectors
                                                 debiased=True, # Debiased estimator
                                                 direct=False # Undirectly debiased
                                                ) 
# Plot
sf_ginibre_box.plot_spectral_estimator(k, s_udtp, 
                                       plot_type="imshow", # Plot type
                                       exact_sf=utils.structure_factor_ginibre, # Exact Ginibre structure factor 
                                       error_bar=True, bins=80, # Regularization of the result
                                       scale="log", # Scale
                                       rasterized=True, # Rasterized the approximation points
                                       label=r"$\widehat{S}$")

## 3.2- Using an isotropic estimator

The isotropic estimators approximate the structure factor of a stationary **isotropic** point process observed in a ``BallWindow``.
As the ``ginibre_pp_ball`` is a ``PointPattern`` having ``BallWindow`` there is no need for any restriction and we can initialize StructureFactor on ``ginibre_pp_ball``.

In [ ]:
sf_ginibre_ball = StructureFactor(ginibre_pp_ball)

### 3.2.1- Bartlett's isotropic estimator

Bartlett's isotropic estimator $\widehat{S}_{\mathrm{BI}}$ is an estimator of the structure factor $S$.
It can be evaluated on arbitrary of allowed wavenumbers (reduce biased).
The complexity of this estimator is quadratic in the number of points of the ``PointPattern``.
When dealing with a large number of points, it is recommended to start with a restricted version of the ``PointPattern``, before using the large sample and increasing the window radius respectively.

In [ ]:
# Bartlett's isotropic estimator on arbitrary wavenumbers
# Running time = 156 s
import time
start_time = time.time()
k_norm = np.linspace(0.01, 7, 100)
k_norm, s_bi = sf_ginibre_ball.bartlett_isotropic_estimator(k_norm=k_norm)
print("--- %s seconds ---" % (time.time() - start_time))

#Plot
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_ball.plot_isotropic_estimator(k_norm, s_bi, 
                                         axis=axis,
                                         exact_sf=utils.structure_factor_ginibre, 
                                         label=r"$\widehat{S}$")

We can see the relevant bias for small wavenumbers

In [ ]:
# Bartlett's isotropic estimator on allowed wavenumbers
# Running time = 156 s
import time
start_time = time.time()
k_norm_allowed, s_bi_k_norm_allowed = sf_ginibre_ball.bartlett_isotropic_estimator(
    n_allowed_k_norm=100 # number of allowed wavenumbers
)
print("--- %s seconds ---" % (time.time() - start_time))

#Plot
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_ball.plot_isotropic_estimator(k_norm_allowed, s_bi_k_norm_allowed, 
                                         axis=axis, # Plot axis
                                         exact_sf=utils.structure_factor_ginibre, # Exact structure factor
                                         label=r"$\widehat{S}$" # Label
                                        )

This estimator seems to give the best results among all the estimators but, it's time-consuming.

### 3.2.2- Estimators using Hankel transform quadrature

The last two isotropic estimators are based on, first estimating the pair correlation function $g$ of the ``PointPattern``,
then interpolating/extrapolating the obtained results, and finally use Ogata/Baddour&Chouinard quadrature to estimate the Hankel transform of a function of g, corresponding to the structure factor.

Let's start by estimating the pair correlation function.

#### 3.2.2.1 - Pair correlation function

**Warning: This section require the R program language https://www.rstudio.com/ to be available on your local machine.**

The toolbox contains two estimators of the pair correlation function (pcf) of a ``PointPattern``.
These two methods are ``pcf.ppp`` and ``pcf.fv`` inherited from the R package ``spatstat`` https://spatstat.org/ .

Approximating the pair correlation function using pcf.ppp

In [ ]:
# pcf.ppp
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

r= np.linspace(0, 20, 200)
ginibre_pcf_ppp = pcf.estimate(ginibre_pp_ball, # PointPattern
                               method="ppp", # Estimation method (could be "ppp" or "fv")
                               r=r, # Estimation radii 
                               correction="all" # Edge correction 
                              )

# Plot
pcf.plot(ginibre_pcf_ppp, 
         exact_pcf=utils.pair_correlation_function_ginibre, # Exact pcf
         figsize=(7,6), # Specific figure size
         color=['grey', 'b', 'darkcyan'], # Colors of the plot
         style=[".", "*", "^"], # Marker style 
        )

Approximating the pair correlation function using pcf.fv

In [ ]:
# pcf.fv
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

ginibre_pcf_fv = pcf.estimate(ginibre_pp_ball, # PointPattern 
                              method="fv", # Estimation method (could be "ppp" or "fv")
                              Kest=dict(rmax=20), # Maximal estimation radius 
                              fv=dict(method="b", spar=0.2) # Correction method and sparsity
                             )

# Plot
pcf.plot(ginibre_pcf_fv, 
         exact_pcf=utils.pair_correlation_function_ginibre, # Exact pcf
         figsize=(7,6), # Figure size
         color=['grey'], # Color
         style=["."] # Marker style
        )

After Approximating the pair correlation function the second step is to clean, interpolate, and extrapolate the result.
We will be working with the approximation result of the method ``pcf.fv``.

In [ ]:
ginibre_pcf_fv

Interpolation

In [ ]:
r = ginibre_pcf_fv["r"]
pcf_r = ginibre_pcf_fv["pcf"]
# Interpolation
ginibre_pcf_fct = pcf.interpolate(r=r, # Radii 
                                  pcf_r=pcf_r, # Approximated pcf
                                  drop=True, # Drop outliers (nan, neginf, and posinf)
                                  extrapolate_with_one=True # Extrapolate with 1 after max(r)
                                 )


# Plot
x = np.linspace(0, 100, 200)
plt.plot(x, ginibre_pcf_fct(x), 'b.', label="Approximated pcf") 
plt.plot(x, utils.pair_correlation_function_ginibre(x), 'g', label="Exact pcf")
plt.legend()
plt.show()

#### 3.2.2.2- Estimate the structure factor using Ogata quadrature

In [ ]:
# Ogata quadrature 
k_norm = np.linspace(0.5, 30, 1000) # Wavenumbers
k_norm, s_ho = sf_ginibre_ball.hankel_quadrature(ginibre_pcf_fct, # Estimated pcf function
                                                 k_norm=k_norm, # wavenumbers
                                                 method="Ogata", # Quadrature method ("Ogata", or "BaddourChouinard")
                                                 step_size=0.01, # Step size
                                                 nb_points=1000 # Number of points
                                                )
# Plot
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre_ball.plot_isotropic_estimator(k_norm, s_ho, 
                                               axis=axis, # Plot axis
                                               error_bar=True, bins=80, # Regularize the results
                                               exact_sf=utils.structure_factor_ginibre, # Exact structure factor 
                                               label=r"$\widehat{S}$",
                                               #file_name="ginibre_s_ho.pdf" # save the figure
                                              )

#### 3.2.2.2- Estimate the structure factor using Baddour & Chouinard quadrature

In [ ]:
# Ogata quadrature 
k_norm = np.linspace(0.5, 30, 1000) # Wavenumbers
r_max = np.max(r) # Maximum radius for which the pcf has been approximated
k_norm, s_hbc = sf_ginibre_ball.hankel_quadrature(ginibre_pcf_fct, # Estimated pcf function
                                                 k_norm=k_norm, # wavenumbers
                                                 method="BaddourChouinard", # Quadrature method ("Ogata", or "BaddourChouinard")
                                                 r_max=r_max, # Step size
                                                 nb_points=1000 # Number of points
                                                )
# Plot
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre_ball.plot_isotropic_estimator(k_norm, s_hbc, 
                                               axis=axis, # Plot axis
                                               #error_bar=True, bins=80, # Regularize the results
                                               exact_sf=utils.structure_factor_ginibre, # Exact structure factor 
                                               label=r"$\widehat{S}$",
                                               #file_name="ginibre_s_hbc.pdf" # save the figure
                                              )

# 3.4- Hyperuniformity

Our strategy for studying the hyperuniformity of a point process is by, first studying the index $H$ of hyperuniformity of the ``PointPattern.
If $H< 10^{-3}$ then the point process is effectively hyperuniform and so maybe hyperuniform. 
In this case, we study the power decay of the structure factor to zero $\alpha$ which indicates the possible class of hyperuniformity.
If the $H$ index rejects the hypothesis of hyperuniformity then we conclude that the point process is not hyperuniform.

In what follows, we will be using the results of Bartlett's isotropic estimator for studying the 
effective hyperuniformity and the hyperuniformity class of the Ginibre point pattern.
We know that the Ginibre ensemble is a first-class hyperuniform point process with power decay $\alpha_{Ginibre} = 2$.


In [ ]:
# Initialize ``Hyperuniformity``
from structure_factor.hyperuniformity import Hyperuniformity

hyperuniformity_test = Hyperuniformity(k_norm_allowed, s_bi_k_norm_allowed)

### 1.3.1- Effective hyperuniformity

In [ ]:
# Effective hyperuniformity
H_ginibre, _ = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=0.2) # H index

# Visualization of the results
import matplotlib.pyplot as plt
import structure_factor.utils as utils
fitted_line = hyperuniformity_test.fitted_line # Fitted line to s_bi
x = np.linspace(0, 2, 300)
fig, axis =plt.subplots(figsize=(7,5))
axis.plot(k_norm_allowed, s_bi_k_norm_allowed, 'b', marker=".", label="Approximated structure factor")
axis.plot(x, fitted_line(x), 'r--', label= "Fitted line")
axis.plot(k_norm_allowed, utils.structure_factor_ginibre(k_norm_allowed), 'g', label=r"$S(k)$")
axis.annotate('H={}'.format(H_ginibre), xy=(0, 0), xytext=(0.01,0.3),
            arrowprops=dict(facecolor='black', shrink=0.01))
axis.legend()
axis.set_xlabel('wavelength (k)')
axis.set_ylabel(r"Structure factor ($\mathsf{S}(k)$")
plt.show()

As $H<10^{-3}$ then we conclude that the Ginibre ensemble is effectively hyperuniformity.

### 1.3.2- Hyperuniformity class

In [ ]:
# Hyperuniformity class
alpha_ginibre, _ = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.4)

# Visualization of the results
import matplotlib.pyplot as plt
import structure_factor.utils as utils
fitted_poly = hyperuniformity_test.fitted_poly # Fitted polynomial to s_bi
fig, axis =plt.subplots(figsize=(7,5))
axis.plot(k_norm_allowed, s_bi_k_norm_allowed, 'b', marker=".", label="Approximated structure factor")
axis.plot(k_norm_allowed, utils.structure_factor_ginibre(k_norm_allowed), 'g', label=r"$S(k)$")
axis.plot(k_norm_allowed, fitted_poly(k_norm_allowed), 'r--', label= "Fitted line")
axis.annotate(r" $\alpha$ ={}".format(alpha_ginibre), xy=(0, 0), xytext=(0.01,0.4),
            arrowprops=dict(facecolor='black', shrink=0.1))
axis.legend()
axis.set_xlabel('wavelength (k)')
axis.set_ylabel(r"Structure factor ($\mathsf{S}(k)$")
plt.show()

This power decay found by this test is $\alpha \approx 1.99$ which is a good approximation of the exact value $\alpha_{Ginibre}=2$.
Thus this test successfully predicts the hyperuniformity class of the Ginibre Ensemble.